In [1]:
import pandas as pd
import pickle 
from scipy.spatial.distance import cosine
from gensim.models import FastText

import sys
sys.path.append('../code')

from preprocessing import GlowpickPreprocessing

# Load data

In [2]:
reviews = pd.read_csv('../dataset/glowpick_reviews.csv')
products = pd.read_csv('../dataset/glowpick_products.csv')
print('reviews.shape: ',reviews.shape)
print('products.shape: ',products.shape)

reviews.shape:  (5208, 7)
products.shape:  (1183, 8)


# Load saved file

In [4]:
text = pickle.load(open('../saved_file/pre_text.pickle','rb'))
text_word = pickle.load(open('../saved_file/pre_text_word.pickle','rb'))
text_pos = pickle.load(open('../saved_file/pre_text_pos.pickle','rb'))
text_word_pos = pickle.load(open('../saved_file/pre_text_word_pos.pickle','rb'))

In [7]:
vec = pickle.load(open('../saved_file/pre_embed.pickle','rb'))
vec_word = pickle.load(open('../saved_file/pre_embed_word.pickle','rb'))
vec_pos = pickle.load(open('../saved_file/pre_embed_pos.pickle','rb'))
vec_word_pos = pickle.load(open('../saved_file/pre_embed_word_pos.pickle','rb'))

In [8]:
model = FastText.load('../saved_file/model.bin')
model_word = FastText.load('../saved_file/model_word.bin')
model_pos = FastText.load('../saved_file/model_pos.bin')
model_word_pos = FastText.load('../saved_file/model_word_pos.bin')

# Test

In [9]:
wordpath = '../saved_file/replace_word.json'
pospath = '../saved_file/pos_dict.json'

In [10]:
search = ['건성 피부에 괜찮은 로션']

In [11]:
GP = GlowpickPreprocessing()
s_text = GP.fit(search)
s_text_word = GP.fit(search, wordfix_path=wordpath)
s_text_pos = GP.fit(search, posfix_path=pospath)
s_text_word_pos = GP.fit(search, wordfix_path=wordpath, posfix_path=pospath)

s_vec = GP.sent2vec(s_text, model)
s_vec_word = GP.sent2vec(s_text_word, model_word)
s_vec_pos = GP.sent2vec(s_text_pos, model_pos)
s_vec_word_pos = GP.sent2vec(s_text_word_pos, model_word_pos)

[Log]: complete stopword
Loading JIT Compiled ChatSpace Model
[Log]: complete spacefix
[Log]: complete stopword
Loading JIT Compiled ChatSpace Model
[Log]: complete spacefix
[Log]: complete wordfix
[Log]: complete stopword
Loading JIT Compiled ChatSpace Model
[Log]: complete spacefix
[Log]: complete posfix
[Log]: complete stopword
Loading JIT Compiled ChatSpace Model
[Log]: complete spacefix
[Log]: complete wordfix
[Log]: complete posfix


# distance

In [12]:
def cosine_dist(m, v):
    dist_arr = np.zeros((m.shape[0]))
    for i, sent in enumerate(m):
        dist = cosine(sent, v[0])
        dist_arr[i] = dist
    return dist_arr

In [13]:
dist = cosine_dist(vec, s_vec)
dist_word = cosine_dist(vec_word, s_vec_word)
dist_pos = cosine_dist(vec_pos, s_vec_pos)
dist_word_pos = cosine_dist(vec_word_pos, s_vec_word_pos)

In [14]:
reviews['dist'] = dist
reviews['dist_word'] = dist_word
reviews['dist_pos'] = dist_pos
reviews['dist_word_pos'] = dist_word_pos

# Filtering

In [15]:
cat_lst = products.category.unique()

In [17]:
reviews_prod = pd.merge(reviews, products, on='product_url', how='left')

rate_dict = {
    'best':0,
    'good':1,
    'soso':2,
    'bad':3,
    'worst':4
}
reviews_prod.rate_x = reviews_prod.rate_x.map(rate_dict)

In [22]:
features = ['category','brand','nb_reviews','sales_rank','sex','age_skin_type','rate_x','content','product']

In [36]:
f = set(s_text[0]) & set(cat_lst)
if len(f) > 0:
    temp = reviews_prod[['dist'] + features]
    temp = temp[temp.category.isin(f)].sort_values(by=['rate_x','dist']).head(20)
else:
    temp = reviews_prod[['dist'] + features].sort_values(by=['rate_x','dist']).head(20)
temp

,dist,category,brand,nb_reviews,sales_rank,sex,age_skin_type,rate_x,content,product
1496,0.000081,로션,불독,(40),4,f,24세 · 중성 ·,0,아부지가 올인원도 좀 건조하다고 하셔서 이번에 이걸 선물해드렸는데 무난무난하게 잘 ...,오리지널 모이스처라이저
1734,0.000087,로션,스킨푸드,(13),14,f,29세 · 복합성 ·,0,남자친구 선물로 좋은듯. 피부를 엄청 신경쓰는 세심한 남자라면 어떨지 잘 모르겠는데...,비어 원샷 수분 밸런서 포맨
1900,0.000091,로션,더페이스샵,(3),28,f,25세 · 복합성 ·,0,이라인쓰고 아빠 피부가 좋아지신 거같은 느낌임.. 좀 걸리는 건 남자 스킨케어 답게...,네오클래식옴므 블랙 에센셜 80에멀젼
1834,0.000093,로션,니베아 맨,(26),19,f,29세 · 중성 ·,0,반신반의 하는 마음으로 구매했고 냄새에 민감한편은 아님! 그랬는데 냄새는 정말 아부...,센서티브 애프터 쉐이브 로션
1895,0.000095,로션,크리니크 (CLINIQUE),(4),27,m,29세 · 지성 ·,0,진짜 크리니크는 지성인나도 피지조절되는게 느껴질정도로 바른후 찝찝함이랄까 번들번들해...,포 맨 모이스춰라이징 로션
1745,0.000103,로션,유키누(UQINU),(9),17,f,33세 · 복합성 ·,0,샘플로 받아서 써보다가 옴므로 나온건데 더마 화장품이라 그런지 의외로 잘맞아서 마침...,더마 모이스트 로션
1729,0.000103,로션,스킨푸드,(13),14,f,39세 · 건성 ·,0,좋은거 세트로 사줘봤자 화장대서 썪히는 남편을 위해 구매. 슼푸 남성용 제품군중 하...,비어 원샷 수분 밸런서 포맨
1923,0.000111,로션,라네즈옴므,(3),34,m,27세 · 지성 ·,0,군대에서 사용한뒤로 쭉 이거 사용하고 있습니다 로션같이 무겁진않고 에멀젼같은느낌이라...,퓨어 브라이트닝 플루이드
1736,0.000116,로션,더페이스샵,(7),16,f,26세 · 중성 ·,0,#사요 #남친선물템 남친이 지성이고 또 건조한 각질도 있는데 이거 선물해줘서 쓰니까...,허브 앤 릴리프 세범 컨트롤 로션
1081,0.000116,로션,입큰,(3),48,f,27세 · 지성 ·,0,엥 뭐지??? 내가 지성이라그런가?? 시간없어서 남자꺼발랐는데 너무 좋앗음 제형은 ...,맨 스타일 퍼펙트 올인원


In [37]:
f = set(s_text[0]) & set(cat_lst)
if len(f) > 0:
    temp = reviews_prod[['dist_word'] + features]
    temp = temp[temp.category.isin(f)].sort_values(by=['rate_x','dist_word']).head(20)
else:
    temp = reviews_prod[['dist_word'] + features].sort_values(by=['rate_x','dist_word']).head(20)
temp

,dist_word,category,brand,nb_reviews,sales_rank,sex,age_skin_type,rate_x,content,product
291,0.000369,로션,반트 36.5 (VANT 36.5),(31),5,m,28세 · 지성 ·,0,지성인데 올인원이라 편하고 피부에도 잘맞아요,포 맨 올 인 원 스킨
1560,0.001151,로션,비오템옴므,(39),6,m,35세 · 복합성 ·,0,요거 물건임 ㅋㅋ 피부가 민감한편인데 이거는 발라도 붉은기가 안올라와서 좋네요,아쿠아파워 건성용
519,0.001675,로션,이니스프리(innisfree),(63),8,f,26세 · 중성 ·,0,피부지성인 울오빠가 쓰는 애정템!! 군대 갈때부터 선물해줘서 쓰고있는데 피부에 잘맞...,포레스트 포맨 프레시 올인원 에센스
1603,0.002502,로션,아이오페,(29),7,m,21세 · 지성 ·,0,이거 써서 피부가 좋아졌습니다 귀차니즘 덜기 좋습니다 촉촉하고 재구매 의사 100%입니다,맨 올데이 퍼펙트 올인원
1956,0.002625,로션,CNP(차앤박),(5),46,f,33세 · 지성 ·,0,우루오스쓰고 피부뒤집어진 남친한테 가성비좋은제품 추천해줘도 이거만 고집하네요.,옴므 에이 케어 로션
609,0.002985,로션,스킨푸드,(27),9,f,26세 · 복합성 ·,0,냄새도 좋고 완전 촉촉하다하는 내 남친님. 피부미남 되실듯,자작나무 수액 수분 플루이드 포맨
1309,0.003004,로션,이니스프리(innisfree),(58),1,f,33세 · 복합성 ·,0,남친이 4년째 쓰고있는제품!! 떨어지면 바로 사고ㅋㅋ향도 향이지만 피부가 진짜 보들...,포레스트 포맨 모이스처 로션
1875,0.003090,로션,더페이스샵,(6),24,f,22세 · 지성 ·,0,아르쌩뜨 라인이 순하고 좋아서 아빠 피부 민감하실 때 선물해드렸어요 가볍지만 보습감...,아르쌩뜨 에코-테라피 포맨 마일드 로션
1088,0.003306,로션,싸이닉,(2),51,f,26세 · 건성 ·,0,#탐사대요청 올인원이라서 너무편하대요! 텍스쳐도 가볍고 촉촉하니 무난한 제품이에요-...,아쿠아 옴므 올인원 모이스처라이저
594,0.003341,로션,스킨푸드,(27),9,f,53세 · 복합성 ·,0,남편을 사줬더니 하나만 발라도되서 잘 사용한다 피부도 예전보다 많이 좋아졌다,자작나무 수액 수분 플루이드 포맨


In [38]:
f = set(s_text[0]) & set(cat_lst)
if len(f) > 0:
    temp = reviews_prod[['dist_pos'] + features]
    temp = temp[temp.category.isin(f)].sort_values(by=['rate_x','dist_pos']).head(20)
else:
    temp = reviews_prod[['dist_pos'] + features].sort_values(by=['rate_x','dist_pos']).head(20)
temp

,dist_pos,category,brand,nb_reviews,sales_rank,sex,age_skin_type,rate_x,content,product
291,0.000117,로션,반트 36.5 (VANT 36.5),(31),5,m,28세 · 지성 ·,0,지성인데 올인원이라 편하고 피부에도 잘맞아요,포 맨 올 인 원 스킨
651,0.000132,로션,시드물,(24),11,f,22세 · 지성 ·,0,오빠가 쓰는걸 봤고 저도 어쩌다보니 한번 우연히 써보게 됐는데 피부가 수분이 촉촉하...,옴므 올인
417,0.000133,로션,이니스프리(innisfree),(63),8,f,22세 · 건성 ·,0,피톤치드 캡슐이 들었다고 하던데 그래서 그런지 아주 약간 화한 느낌이 나면서 시원하...,포레스트 포맨 프레시 올인원 에센스
1722,0.000157,로션,라네즈옴므,(16),13,f,28세 · 건성 ·,0,남자친구가 수분감이 부족해서 사줬는데 너무 좋다그러더라고요~지성인데도 수분라인이라서...,듀얼 액션 에센스 로션
1980,0.000185,로션,피토메스,(4),57,f,30세 · 지성 ·,0,올인원 제품이라서 귀차니즘 심할때 사용해주면 굿굿 하나만 발라도 피부보습이 잘 되고...,블랙 부스터
1969,0.000188,로션,케어존,(3),54,m,25세 · 지성 ·,0,제품 용기가 고급스러워 보였어요 향은 허브향이 났고요 제가 극지성피부여서 유분감이 ...,닥터솔루션 옴므 센서티브 에멀전
1901,0.000207,로션,잇츠스킨 (It'S SKIN),(3),28,m,47세 · 건성 ·,0,예전에 스킨과 크림이 하나에 같이 들어 있는거 쓰다가 이젠 좀 제대로 발라보자 하고...,프레스티지 로씨옹 데스까르고 옴므
513,0.000208,로션,이니스프리(innisfree),(63),8,f,30세 · 민감성 ·,0,제가 쓴 건 아니고 제 남동생이 사용했습니다. 피지도 많고 칙칙한 피부였는데 여러개...,포레스트 포맨 프레시 올인원 에센스
1744,0.000215,로션,유키누(UQINU),(9),17,f,32세 · 복합성 ·,0,남동생에게 선물해줬습니다 종종 트러블 올라오는 복합성 피부에요! 걔 원래 귀찮다며 ...,더마 모이스트 로션
1496,0.000222,로션,불독,(40),4,f,24세 · 중성 ·,0,아부지가 올인원도 좀 건조하다고 하셔서 이번에 이걸 선물해드렸는데 무난무난하게 잘 ...,오리지널 모이스처라이저


In [39]:
f = set(s_text[0]) & set(cat_lst)
if len(f) > 0:
    temp = reviews_prod[['dist_word_pos'] + features]
    temp = temp[temp.category.isin(f)].sort_values(by=['rate_x','dist_word_pos']).head(20)
else:
    temp = reviews_prod[['dist_word_pos'] + features].sort_values(by=['rate_x','dist_word_pos']).head(20)
temp

,dist_word_pos,category,brand,nb_reviews,sales_rank,sex,age_skin_type,rate_x,content,product
291,0.000105,로션,반트 36.5 (VANT 36.5),(31),5,m,28세 · 지성 ·,0,지성인데 올인원이라 편하고 피부에도 잘맞아요,포 맨 올 인 원 스킨
417,0.000131,로션,이니스프리(innisfree),(63),8,f,22세 · 건성 ·,0,피톤치드 캡슐이 들었다고 하던데 그래서 그런지 아주 약간 화한 느낌이 나면서 시원하...,포레스트 포맨 프레시 올인원 에센스
651,0.000138,로션,시드물,(24),11,f,22세 · 지성 ·,0,오빠가 쓰는걸 봤고 저도 어쩌다보니 한번 우연히 써보게 됐는데 피부가 수분이 촉촉하...,옴므 올인
1980,0.000221,로션,피토메스,(4),57,f,30세 · 지성 ·,0,올인원 제품이라서 귀차니즘 심할때 사용해주면 굿굿 하나만 발라도 피부보습이 잘 되고...,블랙 부스터
1969,0.000229,로션,케어존,(3),54,m,25세 · 지성 ·,0,제품 용기가 고급스러워 보였어요 향은 허브향이 났고요 제가 극지성피부여서 유분감이 ...,닥터솔루션 옴므 센서티브 에멀전
513,0.000234,로션,이니스프리(innisfree),(63),8,f,30세 · 민감성 ·,0,제가 쓴 건 아니고 제 남동생이 사용했습니다. 피지도 많고 칙칙한 피부였는데 여러개...,포레스트 포맨 프레시 올인원 에센스
1496,0.000257,로션,불독,(40),4,f,24세 · 중성 ·,0,아부지가 올인원도 좀 건조하다고 하셔서 이번에 이걸 선물해드렸는데 무난무난하게 잘 ...,오리지널 모이스처라이저
534,0.000264,로션,이니스프리(innisfree),(63),8,f,34세 · 건성 ·,0,남편이 트러블성 복합지성피부라서 이번에 바꿔줬는데 세이어스 토너로 피부닦아주고 이 ...,포레스트 포맨 프레시 올인원 에센스
1929,0.000270,로션,애터미,(2),36,f,28세 · 복합성 ·,0,아빠가 좋다고 하심! 지성피부이신데 겨울에 쓰기 좋다네요~,맨 로션
1286,0.000297,로션,이니스프리(innisfree),(58),1,f,27세 · 지성 ·,0,젤 기본적인 성분만 들어가있네용ㅋㅋㅋㅋ 남자친구가 이거 오래썼는데 나보다 피부 훨좋...,포레스트 포맨 모이스처 로션
